In [ ]:
### Install requirements
from os.path import isfile

repository   = "https://github.com/lmingari/olot-course.git"
requirements = "requirements-section1.txt"

if not isfile(requirements):
    !git clone {repository}
    %cd olot-course
    !pip install -r {requirements}

# 1. Loading a FALL3D output file
***

* Results from FALL3D simulations are stored in [netCDF][netcdf] format. 

* __NetCDF (network Common Data Form)__ is an open-source file format and associated software libraries for creating, accessing, and sharing scientific data, particularly multidimensional arrays like meteorological variables

* In order to open and manipulate data from netCDF files, we can use the python package `xarray`. 

* Maps will be created from the model data using `Cartopy`, a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses.

> &#10069; __Note__ <br>
> Some examples of Python scripts for plotting FALL3D outputs can be found in the [FALL3D user guide][fall3d].

[netcdf]: https://www.unidata.ucar.edu/software/netcdf "netCDF"
[fall3d]: https://fall3d-suite.gitlab.io/fall3d/hands-on/introduction.html "FALL3D user guide"

## Data structure: dataset and dataarray

The N-dimensional nature of Xarray’s data structures makes it suitable for dealing with multi-dimensional scientific data, and its use of dimension names instead of axis labels (`dim='time'` instead of `axis=0`) makes such arrays much more manageable.

Here is an example of how we might structure a dataset for a weather forecast:

![](https://docs.xarray.dev/en/stable/_images/dataset-diagram.png)

#### Importing modules

In [ ]:
import xarray as xr                              # multi-dimensional arrays for NetCDF data
import numpy as np                               # numerical operations on arrays
import matplotlib.pyplot as plt                  # plots and visualizations
import cartopy.crs as crs                        # coordinate systems for maps
import cartopy.feature as cfeature

#### Loading data and inspecting file structure

In [ ]:
from os.path import isfile

## Get a FALL3D output file
fname = "data/olot_20251015.res.nc"
if not isfile(fname):
    !wget -P ./data/ https://saco.csic.es/s/jLCZZjL56dfeGtB/download/olot_20251015.res.nc

In [ ]:
## Open a netCDF file in a xarray dataset
fname = "data/olot_20251015.res.nc"
ds = xr.open_dataset(fname)

# Let's see the structure of the data
ds

#### Selecting variables and indexing

In [ ]:
# Selecting a single variable
da = ds['tephra_col_mass']
da

In [ ]:
# Indexing
da = ds['tephra_col_mass'].isel(time=-1)
da

In [ ]:
da.time

## Plotting geodata with cartopy

#### Creating an empty map

In [ ]:
from helper_plot import create_map
fig, ax = create_map()

In [ ]:
fig, ax = create_map()
###
### Set map limits
###
ax.set_extent([-10, 15, 30, 50]) # [x1,x2,y1,y2]
###
### Volcà del Montsacopa
####
vlat, vlon = 42.187193, 2.488525
###
### Add vent location
###
ax.plot(vlon,vlat,color='red',marker='^')

#### Plotting filled contours

In [ ]:
fig, ax = create_map()

###
### Volcà del Montsacopa
####
vlat, vlon = 42.187193, 2.488525

fc = ax.contourf(
    da.lon,da.lat,da,
    levels    = [0.1, 0.5, 1, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0],
    cmap      = plt.cm.RdYlBu_r,
    extend    = 'max',
    )

cbar=fig.colorbar(
    fc, 
    orientation = 'horizontal',
    label = r'Tephra column mass [$g~m^{-2}$]',
    shrink = 0.5,                  
)
ax.set(title='FALL3D output')

## Working with ensemble data

#### Probabilistic dispersion modelling of an eruption at La Palma with FALL3D

In [ ]:
from os.path import isfile

## Get a FALL3D ensemble run output
fname = "data/tephra_col_mass.ens.nc"
if not isfile(fname):
    !wget -P ./data https://saco.csic.es/s/wFpKYG5bHfTwKbi/download/tephra_col_mass.ens.nc

In [ ]:
# Open a netCDF file in a xarray dataset
ds = xr.open_dataset(fname)
da = ds['tephra_col_mass']
da

#### Exceedance probabilities

Given an ensemble of column mass forecasts, let's:
* Compute exceedance probabilities for an intensity threshold of $10~g/m^2$
* Plot contour lines for probabilities of 2%, 25%, 50%, 75%, 98%

In [ ]:
## Compute the probability of exceeding 10 g/m2
threshold = 10
probabilities = 100*(da>threshold).mean(dim='ens')
probabilities

In [ ]:
## Create a probability contour plot
fig, ax = create_map()

cs = ax.contour(
    da.lon,da.lat,probabilities,
    levels    = [2, 25, 50, 75, 98],
    #cmap      = plt.cm.RdYlBu_r,
    #extend    = 'max',
    #transform = proj
    )
ax.clabel(cs, cs.levels, inline = False, fontsize=10)
ax.set_extent([-22, -11, 23.5, 29]) # [x1,x2,y1,y2]
ax.set(title='Exceedance probabilities')

> &#9998; **Exercise:** <br>
> Compute and plot the ensemble mean for the tephra column mass